## Module Import 

In [ ]:
import sys
import math
import numpy as np
import matplotlib.pyplot as plt
import h5py
import re
import os
import time as t
import pandas as pd

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score,mean_squared_error
from sklearn.preprocessing import MinMaxScaler

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Activation
from tensorflow.python.keras.layers import Dense
np.set_printoptions(precision=8)
seed=99

print(tf. __version__)

2.7.0


In [ ]:
# change the directory to data location
from google.colab import drive
drive.mount('/content/drive')
os.chdir('/content/drive/My Drive/Colab Notebooks/PINN')

Mounted at /content/drive


In [ ]:
# Read the file
file = 'major_chemical_marker.h5'
f = h5py.File(file, 'r')

In [ ]:
# Get the feature and the label
# The dataset have larger points HHR region [data= Augumentation]
dataset = np.array(f.get('major_chemical_marker'))
print(dataset.shape)

(51966851, 5)


In [ ]:
# Separate the feature and the label
X = dataset[:,:-1]
Y = dataset[:,[-1]]
#print(X,Y.shape)

In [ ]:
# max and min value 
print("max value of 'nh' in the overall dataset: ",np.max(X[:,0:1]))
print("min value of 'nh' in the overall dataset: ",np.min(X[:,0:1]))
print("max value of 'nh2' in the overall dataset: ",np.max(X[:,1:2]))
print("min value of 'nh2' in the overall dataset: ",np.min(X[:,1:2]))
print("max value of 'oh' in the overall dataset: ",np.max(X[:,2:3]))
print("min value of 'oh' in the overall dataset: ",np.min(X[:,2:3]))
print("max value of 'temp' in the overall dataset: ",np.max(X[:,3:4]))
print("min value of 'temp' in the overall dataset: ",np.min(X[:,3:4]))
print("max value of 'HRR' in the overall dataset: ",np.max(Y))
print("min value of 'HRR' in the overall dataset: ",np.min(Y))

NameError: ignored

In [ ]:
#shuffle the dataset
X,Y = shuffle(X, Y, random_state=seed)
print(X, Y)

[[4.0471452e-05 2.2109612e-04 3.7558284e-03 1.9944869e+03]
 [3.5728943e-05 2.3027172e-04 3.7748790e-03 1.7902939e+03]
 [9.2750714e-05 1.6765213e-03 6.4110372e-04 1.4570482e+03]
 ...
 [1.5075237e-04 2.1670584e-03 1.0331237e-03 1.5530289e+03]
 [2.4906921e-04 1.7336031e-03 4.0482101e-03 1.8717084e+03]
 [1.6094409e-05 6.5804902e-04 1.0320299e-04 1.4690447e+03]] [[3.7575190e+08]
 [9.2389549e+08]
 [3.8451505e+09]
 ...
 [6.0870994e+09]
 [8.8248003e+09]
 [2.3323902e+08]]


In [ ]:
# create the normalization fit 
scale_X = MinMaxScaler()
scale_Y = MinMaxScaler()

# The fit function
print(scale_X.fit(X))
print(scale_Y.fit(Y))

MinMaxScaler()
MinMaxScaler()


In [ ]:
#test_train split 
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.1)
print(X_train.shape, X_test.shape, Y_train.shape, Y_test.shape)

(46770165, 4) (5196686, 4) (46770165, 1) (5196686, 1)


In [ ]:
# normalize the train data and label
X_train_n = tf.Variable(scale_X.transform(X_train))
Y_train_n = tf.Variable(scale_Y.transform(Y_train))
print(X_train_n)
print(Y_train_n)

<tf.Variable 'Variable:0' shape=(46770165, 4) dtype=float32, numpy=
array([[0.31382945, 0.65674746, 0.05627484, 0.39231658],
       [0.39019996, 0.2819212 , 0.32261512, 0.81065136],
       [0.5840067 , 0.39694157, 0.296873  , 0.7851749 ],
       ...,
       [0.34780934, 0.21417964, 0.569124  , 0.7503266 ],
       [0.12475727, 0.42953947, 0.015095  , 0.22589856],
       [0.03380108, 0.12464201, 0.02742347, 0.5030305 ]], dtype=float32)>
<tf.Variable 'Variable:0' shape=(46770165, 1) dtype=float32, numpy=
array([[0.40313682],
       [0.14265038],
       [0.3429449 ],
       ...,
       [0.34454247],
       [0.19807889],
       [0.00869274]], dtype=float32)>


In [ ]:
# Create the chunk 
X_chunk = tf.Variable(X_train_n[0:100000,:])
Y_chunk = tf.Variable(Y_train_n[0:100000,:])
print(X_chunk, Y_chunk)

<tf.Variable 'Variable:0' shape=(100000, 4) dtype=float32, numpy=
array([[0.31382945, 0.65674746, 0.05627484, 0.39231658],
       [0.39019996, 0.2819212 , 0.32261512, 0.81065136],
       [0.5840067 , 0.39694157, 0.296873  , 0.7851749 ],
       ...,
       [0.60118645, 0.447825  , 0.37877324, 0.72503597],
       [0.3885187 , 0.23752129, 0.30629912, 0.9158283 ],
       [0.30058613, 0.63581985, 0.04304647, 0.5448019 ]], dtype=float32)> <tf.Variable 'Variable:0' shape=(100000, 1) dtype=float32, numpy=
array([[0.40313682],
       [0.14265038],
       [0.3429449 ],
       ...,
       [0.5344882 ],
       [0.0898007 ],
       [0.17860411]], dtype=float32)>


In [ ]:
# Creaate the batch slices.
train_data = tf.data.Dataset.from_tensor_slices((X_chunk, Y_chunk))
train_data = train_data.batch(1024, drop_remainder=True)
print(train_data)
# test_data = tf.data.Dataset.from_tensor_slices((X_test, Y_test))
# test_data = test_data.batch(1024, drop_remainder=True)

<BatchDataset shapes: ((1024, 4), (1024, 1)), types: (tf.float32, tf.float32)>


In [ ]:
# Get the pretrained model
# Read the file
file = 'model_250_um.h5'
model = tf.keras.models.load_model(file)

# Get summary
model.summary()

Model: "Heat_release_rate"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 h_layer1 (Dense)            (None, 40)                200       
                                                                 
 h_layer2 (Dense)            (None, 24)                984       
                                                                 
 h_layer3 (Dense)            (None, 24)                600       
                                                                 
 output_layers (Dense)       (None, 1)                 25        
                                                                 
Total params: 1,809
Trainable params: 1,809
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# The Ki the is respective exponents of the species massfraction and temperature, which could yeild us a perfect reconstruction of the HRR
k1,k2,k3,k4,k5 = tf.Variable(-1.8),tf.Variable(-1.6),tf.Variable(-1.4),tf.Variable(-1.5),tf.Variable(1.3)

In [ ]:
# The customized mean square error, taking the batch data at any instance
def customized_MSE(true, pred):
    MSE = tf.reduce_mean(tf.square(true-pred))
    return MSE
# The customized loss taking the non-linear equation as residual
def total_mse(MSE_NN, x, y, k1, k2, k3, k4, k5):# the MSE_NN is the mean squared error of the neural network 
    residual = tf.square(tf.math.log(y) - (k1 * tf.math.log(x[:,0:1]) + k2 * tf.math.log(x[:,1:2]) + k3 * tf.math.log(x[:,2:3]) + k4 * tf.math.log(x[:,3:4]) + tf.math.log(k5)))
    MSE_func = tf.reduce_mean(residual)
    total =  MSE_NN + MSE_func
    return total

In [ ]:
# The optimizer
optimizer= tf.keras.optimizers.SGD(learning_rate=0.3,momentum=0.8)

In [ ]:
#Training with customized optimization technic (Automatic-Gradient)
epochs = 50
lr_rate = 0.03
n_loss = []

for epoch in range(epochs):
  print("Starting a new epoch",epoch+1)
  for step, (X_batch, Y_batch) in enumerate(train_data):
    with tf.GradientTape(persistent=True) as tape:
      Y_pred = model(X_batch)
      current_loss = customized_MSE(Y_batch,Y_pred)
      new_loss = total_mse(current_loss, X_batch, Y_batch, k1, k2, k3, k4, k5)

    # The gradient check  
    gradient = tape.gradient(new_loss, [k1,k2,k3,k4,k5])
    grad = tape.gradient(current_loss, model.trainable_variables)

    #Assign new variables to the model using optimizer isntead of sub assign
    optimizer.apply_gradients(zip(grad, model.trainable_variables))

    #Update the Ki to reduce the residual
    k1.assign_sub(gradient[0]*lr_rate)
    k2.assign_sub(gradient[1]*lr_rate)
    k3.assign_sub(gradient[2]*lr_rate)
    k4.assign_sub(gradient[3]*lr_rate)
    k5.assign_sub(gradient[4]*lr_rate)
  
  print("k1:{} k2:{} k3:{} k4:{} k5:{}".format(k1.numpy(),k2.numpy(),k3.numpy(),k4.numpy(), k5.numpy()))
  print("current_Loss: {} new_loss: {}".format(current_loss.numpy(), new_loss.numpy()))  

Starting a new epoch 1
k1:0.5567150712013245 k2:0.41947028040885925 k3:0.4588121771812439 k4:-1.1722408533096313 k5:0.8355854153633118
current_Loss: 2.1449293853947893e-05 new_loss: 0.1864970475435257
Starting a new epoch 2
k1:0.5700356364250183 k2:0.43994614481925964 k3:0.5352523922920227 k4:-1.3938355445861816 k5:0.9087035059928894
current_Loss: 2.144658355973661e-05 new_loss: 0.16707271337509155
Starting a new epoch 3
k1:0.5679101347923279 k2:0.4602029025554657 k3:0.5971948504447937 k4:-1.5738235712051392 k5:0.9494304656982422
current_Loss: 2.144381869584322e-05 new_loss: 0.1557900756597519
Starting a new epoch 4
k1:0.5636575818061829 k2:0.4776991605758667 k3:0.6472644209861755 k4:-1.7183778285980225 k5:0.9805660843849182
current_Loss: 2.1441108401631936e-05 new_loss: 0.14926691353321075
Starting a new epoch 5
k1:0.5593042373657227 k2:0.4925301969051361 k3:0.6877524852752686 k4:-1.8342067003250122 k5:1.0064525604248047
current_Loss: 2.1438394469441846e-05 new_loss: 0.145658299326896